In [203]:
# Libraries
import os
import pandas as pd
import re as re

# Docker only (setting the path using the environmental variable PROJ_LIB)
#if platform.system() == 'Linux':
#    import os
#    import conda
#    conda_file_dir = conda.__file__
#    conda_dir = conda_file_dir.split('lib')[0]
#    proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
#    os.environ["PROJ_LIB"] = proj_lib
#else:
#    pass

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [204]:
# Read Airbnb Data
airbnb = pd.read_csv('../data/airbnb_apartments_scraped_security_copy.csv',dtype='string')
airbnb.head(10)

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.40581,8.55173,36 Bewertungen,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40 CHF,<NA>,"Uitikon, Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.36336,8.46183,<NA>,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48 CHF,<NA>,"Schlieren, Zürich, Schweiz",1 Gast,1 Schlafzimmer,1 Bett,1 Badezimmer,47.3951,8.4687,10 Bewertungen,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62 CHF,<NA>,"Zürich, Schweiz",1 Gast,1 Schlafzimmer,1 Bett,1 Badezimmer,47.38157,8.48096,341 Bewertungen,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60 CHF,<NA>,"Affoltern am Albis, Zürich, Schweiz",3 Gäste,1 Schlafzimmer,2 Betten,1 Badezimmer,47.27565,8.45244,160 Bewertungen,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."
5,1stFloor Apartment with Mattress,70 CHF,<NA>,"Zollikon, Zürich, Schweiz",3 Gäste,2 Schlafzimmer,1 Bett,1 Badezimmer,47.3427,8.6054,<NA>,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Reinigungsprodukte', 'Warmwasse..."
6,Spannende Wohnung in Zürich Örlikon,62 CHF,<NA>,"Zürich, Schweiz",2 Gäste,Studio,1 Bett,1 Badezimmer,47.40388,8.54967,3 Bewertungen,"4,67 ·","5,0","5,0","5,0","5,0","4,7","5,0",['The house is perfectly connected to the cent...,"['Warmwasser', 'Waschmaschine', 'Trockner', 'G..."
7,Gemütliche Ferienwohnung bang im Zentrum von H...,59 CHF,53 CHF,"Horgen, Zürich, Schweiz",2 Gäste,1 Schlafzimmer,2 Betten,1 Badezimmer,47.26012,8.5973,40 Bewertungen,"4,65 ·","4,7","5,0","4,9","4,8","4,8","4,8","['So schön, unkompliziert und gemütlich - ich ...","['Seeblick', 'Warmwasser', 'Waschmaschine\xa0–..."
8,Geschätzt einfache Wohnung,66 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.4014,8.499,<NA>,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Föhn', 'Reinigungsprodukte', 'S..."
9,Wil ZH Zuhause mit Aussicht,62 CHF,<NA>,"Wil, Zürich, Schweiz",2 Gäste,Studio,1 Bett,1 Badezimmer,47.60506,8.50563,45 Bewertungen,"4,80 ·","4,7","4,7","5,0","4,9","4,9","4,8","['Sehr schönes Appartement, Wifi, gute Erkläru...","['Seife/Duschgel der Marke Palmolive', 'Warmwa..."


In [205]:
#check value types
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  960 non-null    string
 1   price                 948 non-null    string
 2   price_reduced         93 non-null     string
 3   place                 859 non-null    string
 4   guests                960 non-null    string
 5   bedrooms              960 non-null    string
 6   beds                  960 non-null    string
 7   bathrooms             949 non-null    string
 8   lat                   960 non-null    string
 9   lng                   960 non-null    string
 10  nr_of_ratings         641 non-null    string
 11  rating_overall        752 non-null    string
 12  rating_valueformoney  634 non-null    string
 13  rating_location       634 non-null    string
 14  rating_checkin        634 non-null    string
 15  rating_communication  634 non-null    st

In [206]:
airbnb.isnull().sum()

name                      0
price                    12
price_reduced           867
place                   101
guests                    0
bedrooms                  0
beds                      0
bathrooms                11
lat                       0
lng                       0
nr_of_ratings           319
rating_overall          208
rating_valueformoney    326
rating_location         326
rating_checkin          326
rating_communication    326
rating_cleanliness      326
rating_data_accuracy    326
comments                  0
list_ameneties            0
dtype: int64

In [207]:
# remove flats that have no price and no reduced_price
airbnb = airbnb[~((airbnb['price'] == '<NA>') & (airbnb['price_reduced'] == '<NA>'))]
airbnb.loc[airbnb['price'].isnull()]


,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties


In [208]:
airbnb.isna().sum()

name                      0
price                     0
price_reduced           855
place                    99
guests                    0
bedrooms                  0
beds                      0
bathrooms                11
lat                       0
lng                       0
nr_of_ratings           307
rating_overall          206
rating_valueformoney    314
rating_location         314
rating_checkin          314
rating_communication    314
rating_cleanliness      314
rating_data_accuracy    314
comments                  0
list_ameneties            0
dtype: int64

In [209]:
# clean special case where bathrooms is in beds column
def update_bathroom_rows(df):
    def check_and_update(row):
        if row['beds'] == '1 Badezimmer' and row['guests'] == '2 Gäste':
            row['beds'] = '1'
            row['bathrooms'] = '1'
        return row

    return df.apply(check_and_update, axis=1)

airbnb = update_bathroom_rows(airbnb)
airbnb.head(63)

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.40581,8.55173,36 Bewertungen,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40 CHF,<NA>,"Uitikon, Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.36336,8.46183,<NA>,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48 CHF,<NA>,"Schlieren, Zürich, Schweiz",1 Gast,1 Schlafzimmer,1 Bett,1 Badezimmer,47.3951,8.4687,10 Bewertungen,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62 CHF,<NA>,"Zürich, Schweiz",1 Gast,1 Schlafzimmer,1 Bett,1 Badezimmer,47.38157,8.48096,341 Bewertungen,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60 CHF,<NA>,"Affoltern am Albis, Zürich, Schweiz",3 Gäste,1 Schlafzimmer,2 Betten,1 Badezimmer,47.27565,8.45244,160 Bewertungen,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1.5 Zimmer Wohnung in Zürich-Stadt (DE-BE23),77 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.37434,8.51516,8 Bewertungen,"4,0 ·","3,6","4,9","4,5","4,5","4,5","4,1","['Erstaunliche kleine Unterkunft für das, was ...","['Föhn', 'Shampoo', 'Seife oder Duschgel', 'Wa..."
59,1 Large Studio Flat in Zurich/ Kreis 9,90 CHF,<NA>,"Zürich, Schweiz",1 Gast,1 Schlafzimmer,1 Bett,1 Badezimmer,47.3808,8.4957,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Bequeme Wohnung und Angela war sehr reaktion...,"['Blick in den Garten', 'Waschmaschine', 'TV',..."
60,1.5 Zimmerwohnung in Zürich (DE-BE22),74 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.37601,8.51647,<NA>,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Föhn', 'Reinigungsprodukte', 'S..."
61,Urbane Wohnung im Zürcher Seefeld,85 CHF,<NA>,"Zürich, Schweiz",2 Gäste,1 Schlafzimmer,1 Bett,1 Badezimmer,47.3568,8.55211,10 Bewertungen,"4,90 ·","5,0","5,0","5,0","4,9","5,0","5,0","['Dominik is amazing host, responsive and very...","['Föhn', 'Waschmaschine', 'Trockner', 'Grundau..."


In [238]:
def remove_chf_and_spaces(text):
    text_without_chf = re.sub('CHF', '', str(text))
    text_without_spaces = re.sub('\s+', '', text_without_chf)
    return text_without_spaces

# remove ' CHF' from price
airbnb['price'] = airbnb['price'].apply(remove_chf_and_spaces)
airbnb['price'] = airbnb['price'].astype(float)
airbnb.head(50)



,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44.0,0,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,36,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40.0,0,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,0,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,0,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,10,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62.0,0,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,341,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,0,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,160,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."
5,1stFloor Apartment with Mattress,70.0,0,"Zollikon, Zürich, Schweiz",3,2,1,2,47.3427,8.6054,0,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Reinigungsprodukte', 'Warmwasse..."
6,Spannende Wohnung in Zürich Örlikon,62.0,0,"Zürich, Schweiz",2,1,1,2,47.40388,8.54967,3,"4,67 ·","5,0","5,0","5,0","5,0","4,7","5,0",['The house is perfectly connected to the cent...,"['Warmwasser', 'Waschmaschine', 'Trockner', 'G..."
7,Gemütliche Ferienwohnung bang im Zentrum von H...,59.0,53 CHF,"Horgen, Zürich, Schweiz",2,1,2,2,47.26012,8.5973,40,"4,65 ·","4,7","5,0","4,9","4,8","4,8","4,8","['So schön, unkompliziert und gemütlich - ich ...","['Seeblick', 'Warmwasser', 'Waschmaschine\xa0–..."
8,Geschätzt einfache Wohnung,66.0,0,"Zürich, Schweiz",2,1,1,2,47.4014,8.499,0,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Föhn', 'Reinigungsprodukte', 'S..."
9,Wil ZH Zuhause mit Aussicht,62.0,0,"Wil, Zürich, Schweiz",2,1,1,2,47.60506,8.50563,45,"4,80 ·","4,7","4,7","5,0","4,9","4,9","4,8","['Sehr schönes Appartement, Wifi, gute Erkläru...","['Seife/Duschgel der Marke Palmolive', 'Warmwa..."


In [225]:
# function to only keep number and remove text

def remove_chars_keep_numbers(text):
    try:
        numbers_only = [char for char in str(text) if char.isdigit()]
        returnString = ''.join(numbers_only)
        if returnString == '':
            return '0'
        return ''.join(numbers_only)
    except:
        print('remove_chars_keep_numbers error: ' + str(text))
        return str(text)
    

In [226]:
# remove chars in guests column
airbnb['guests'] = airbnb['guests'].apply(remove_chars_keep_numbers)
airbnb['guests'] = airbnb['guests'].astype(int)
airbnb.head()

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,36,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40.0,<NA>,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,<NA>,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,10,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62.0,<NA>,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,341,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,<NA>,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,160,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."


In [227]:
def clean_bedroom_vals(text):
    if str(text) == 'Studio':
        return '1'
    if str(text) == '':
        return '0'
    # remove chars in bedrooms column
    return remove_chars_keep_numbers(str(text))

airbnb['bedrooms'] = airbnb['bedrooms'].apply(clean_bedroom_vals)
airbnb['bedrooms'] = airbnb['bedrooms'].astype(int)
airbnb.head(10)

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,36,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40.0,<NA>,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,<NA>,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,10,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62.0,<NA>,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,341,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,<NA>,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,160,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."
5,1stFloor Apartment with Mattress,70.0,<NA>,"Zollikon, Zürich, Schweiz",3,2,1,2,47.3427,8.6054,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Reinigungsprodukte', 'Warmwasse..."
6,Spannende Wohnung in Zürich Örlikon,62.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40388,8.54967,3,"4,67 ·","5,0","5,0","5,0","5,0","4,7","5,0",['The house is perfectly connected to the cent...,"['Warmwasser', 'Waschmaschine', 'Trockner', 'G..."
7,Gemütliche Ferienwohnung bang im Zentrum von H...,59.0,53 CHF,"Horgen, Zürich, Schweiz",2,1,2,2,47.26012,8.5973,40,"4,65 ·","4,7","5,0","4,9","4,8","4,8","4,8","['So schön, unkompliziert und gemütlich - ich ...","['Seeblick', 'Warmwasser', 'Waschmaschine\xa0–..."
8,Geschätzt einfache Wohnung,66.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.4014,8.499,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Föhn', 'Reinigungsprodukte', 'S..."
9,Wil ZH Zuhause mit Aussicht,62.0,<NA>,"Wil, Zürich, Schweiz",2,1,1,2,47.60506,8.50563,45,"4,80 ·","4,7","4,7","5,0","4,9","4,9","4,8","['Sehr schönes Appartement, Wifi, gute Erkläru...","['Seife/Duschgel der Marke Palmolive', 'Warmwa..."


In [228]:
# remove chars in beds column
airbnb['beds'] = airbnb['beds'].apply(remove_chars_keep_numbers)
airbnb['beds'] = airbnb['beds'].astype(int)

In [229]:
def clean_bathroom_vals(text):
    text = str(text)
    if text == '1 Badezimmer' or text == 'WC und Waschbecken':
        return '1'
    if text == '0 Badezimmer' or text == '':
        return '0'
    if text == '2 Badezimmer' or text == '1,5 Bäder' or '1,5 Badezimmer':
        return '2'
    if text == '3 Badezimmer' or text == '2,5 Bäder' or '2,5 Badezimmer':
        return '3'
    if text == '4 Badezimmer' or text == '3,5 Bäder' or '3,5 Badezimmer':
        return '4'
    if text == '5 Badezimmer' or text == '4,5 Bäder' or '4,5 Badezimmer':
        return '5'
    return text


# remove chars in bathrooms column
airbnb['bathrooms'].astype(str)
airbnb['bathrooms'] = airbnb['bathrooms'].apply(clean_bathroom_vals)
airbnb['bathrooms'] = airbnb['bathrooms'].astype(int)
airbnb.head(20)

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,36,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40.0,<NA>,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,<NA>,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,10,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62.0,<NA>,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,341,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,<NA>,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,160,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."
5,1stFloor Apartment with Mattress,70.0,<NA>,"Zollikon, Zürich, Schweiz",3,2,1,2,47.3427,8.6054,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Reinigungsprodukte', 'Warmwasse..."
6,Spannende Wohnung in Zürich Örlikon,62.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40388,8.54967,3,"4,67 ·","5,0","5,0","5,0","5,0","4,7","5,0",['The house is perfectly connected to the cent...,"['Warmwasser', 'Waschmaschine', 'Trockner', 'G..."
7,Gemütliche Ferienwohnung bang im Zentrum von H...,59.0,53 CHF,"Horgen, Zürich, Schweiz",2,1,2,2,47.26012,8.5973,40,"4,65 ·","4,7","5,0","4,9","4,8","4,8","4,8","['So schön, unkompliziert und gemütlich - ich ...","['Seeblick', 'Warmwasser', 'Waschmaschine\xa0–..."
8,Geschätzt einfache Wohnung,66.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.4014,8.499,,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Badewanne', 'Föhn', 'Reinigungsprodukte', 'S..."
9,Wil ZH Zuhause mit Aussicht,62.0,<NA>,"Wil, Zürich, Schweiz",2,1,1,2,47.60506,8.50563,45,"4,80 ·","4,7","4,7","5,0","4,9","4,9","4,8","['Sehr schönes Appartement, Wifi, gute Erkläru...","['Seife/Duschgel der Marke Palmolive', 'Warmwa..."


In [230]:
# remove chars in nr_of_ratings column
airbnb['nr_of_ratings'] = airbnb['nr_of_ratings'].apply(remove_chars_keep_numbers)
airbnb['nr_of_ratings'] = airbnb['nr_of_ratings'].astype(int)
airbnb.head(5)

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties
0,Serviced Studio Zürich City #B,44.0,<NA>,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,36,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei..."
1,2.5 Zimmerwohnung mit Garten,40.0,<NA>,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,0,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma..."
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,<NA>,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,10,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun..."
3,Stadt Zürich15/ kleines Studio,62.0,<NA>,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,341,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ..."
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,<NA>,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,160,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei..."


In [253]:
# function to replace <NA> by 0
def change_na_to_0(text: str):
    if str(text) == '<NA>':
        return '0'
    return str(text)

In [247]:
# clean price_reduced
airbnb['price_reduced'] = airbnb['price_reduced'].apply(change_na_to_0)
airbnb['price_reduced'] = airbnb['price_reduced'].apply(remove_chf_and_spaces)
airbnb['price_reduced'] = airbnb['price_reduced'].astype(int)
airbnb.head()

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,...,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties,priprice_reducedce
0,Serviced Studio Zürich City #B,44.0,0,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,...,"4,28 ·",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei...",0
1,2.5 Zimmerwohnung mit Garten,40.0,0,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,...,Neu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma...",0
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,0,"Schlieren, Zürich, Schweiz",1,1,1,2,47.3951,8.4687,...,"4,10 ·","4,7","4,4","4,7","4,6","4,0","4,6","['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun...",0
3,Stadt Zürich15/ kleines Studio,62.0,0,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,...,"4,87 ·","4,8","4,7","5,0","4,9","4,9","4,9",['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ...",0
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,0,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,...,"4,53 ·","4,6","4,7","4,9","4,8","4,3","4,8","['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei...",0


In [251]:
airbnb['lat'] = airbnb['lat'].astype(float)
airbnb['lng'] = airbnb['lng'].astype(float)
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 948 entries, 0 to 959
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  948 non-null    string 
 1   price                 948 non-null    object 
 2   price_reduced         948 non-null    int64  
 3   place                 849 non-null    string 
 4   guests                948 non-null    int64  
 5   bedrooms              948 non-null    int64  
 6   beds                  948 non-null    int64  
 7   bathrooms             948 non-null    int64  
 8   lat                   948 non-null    float64
 9   lng                   948 non-null    float64
 10  nr_of_ratings         948 non-null    int64  
 11  rating_overall        742 non-null    string 
 12  rating_valueformoney  634 non-null    string 
 13  rating_location       634 non-null    string 
 14  rating_checkin        634 non-null    string 
 15  rating_communication  6

In [268]:
# replace <NA> by 0
airbnb['rating_overall'] = airbnb['rating_overall'].apply(change_na_to_0)
# Set all sub-ratings to 5.0 if rating_overall is "Superhost"
def update_superhost_rows(df):
    def check_and_update(row):
        if row['rating_overall'] == 'Superhost':
            row['rating_overall'] = '5'
            row['rating_valueformoney'] = '5'
            row['rating_location'] = '5'
            row['rating_checkin'] = '5'
            row['rating_communication'] = '5'
            row['rating_cleanliness'] = '5'
            row['rating_data_accuracy'] = '5'
        return row

    return df.apply(check_and_update, axis=1)

airbnb = update_superhost_rows(airbnb)

# Convert 'Neu' to zero and remove ' ·' from rating
def clean_rating_overall(text: str):
    if str(text) == 'Neu' or str(text) == '':
        return '0'
    return str(text).replace(' ·', '').replace(',','.')

# rating_overall
airbnb['rating_overall'] = airbnb['rating_overall'].apply(clean_rating_overall)
airbnb['rating_overall'] = airbnb['rating_overall'].astype(float)
#rating_valueformoney
airbnb['rating_valueformoney'] = airbnb['rating_valueformoney'].apply(change_na_to_0)
airbnb['rating_valueformoney'] = airbnb['rating_valueformoney'].apply(clean_rating_overall)
airbnb['rating_valueformoney'] = airbnb['rating_valueformoney'].astype(float)
#rating_location
airbnb['rating_location'] = airbnb['rating_location'].apply(change_na_to_0)
airbnb['rating_location'] = airbnb['rating_location'].apply(clean_rating_overall)
airbnb['rating_location'] = airbnb['rating_location'].astype(float)
#rating_checkin
airbnb['rating_checkin'] = airbnb['rating_checkin'].apply(change_na_to_0)
airbnb['rating_checkin'] = airbnb['rating_checkin'].apply(clean_rating_overall)
airbnb['rating_checkin'] = airbnb['rating_checkin'].astype(float)
#rating_communication
airbnb['rating_communication'] = airbnb['rating_communication'].apply(change_na_to_0)
airbnb['rating_communication'] = airbnb['rating_communication'].apply(clean_rating_overall)
airbnb['rating_communication'] = airbnb['rating_communication'].astype(float)
#rating_cleanliness
airbnb['rating_cleanliness'] = airbnb['rating_cleanliness'].apply(change_na_to_0)
airbnb['rating_cleanliness'] = airbnb['rating_cleanliness'].apply(clean_rating_overall)
airbnb['rating_cleanliness'] = airbnb['rating_cleanliness'].astype(float)
#rating_data_accuracy
airbnb['rating_data_accuracy'] = airbnb['rating_data_accuracy'].apply(change_na_to_0)
airbnb['rating_data_accuracy'] = airbnb['rating_data_accuracy'].apply(clean_rating_overall)
airbnb['rating_data_accuracy'] = airbnb['rating_data_accuracy'].astype(float)
airbnb.head()

,name,price,price_reduced,place,guests,bedrooms,beds,bathrooms,lat,lng,...,rating_overall,rating_valueformoney,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,comments,list_ameneties,priprice_reducedce
0,Serviced Studio Zürich City #B,44.0,0,"Zürich, Schweiz",2,1,1,2,47.40581,8.55173,...,4.28,0.0,0.0,0.0,0.0,0.0,0.0,['Zum Übernachten optimal für eine Person.\nFü...,"['Grundausstattung', 'Heizung', 'WLAN', 'Arbei...",0
1,2.5 Zimmerwohnung mit Garten,40.0,0,"Uitikon, Zürich, Schweiz",2,1,1,2,47.36336,8.46183,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,[],"['Seife oder Duschgel', 'Warmwasser', 'Waschma...",0
2,Schönes Reihenhaus in Schlieren Altstiel.,48.0,0,"Schlieren, Zürich, Schweiz",1,1,1,2,47.39510,8.46870,...,4.10,4.7,4.4,4.7,4.6,4.0,4.6,"['Hallo zusammen,\nHerr Borotti hatte mich tro...","['Blick in den Garten', 'Badewanne', 'Reinigun...",0
3,Stadt Zürich15/ kleines Studio,62.0,0,"Zürich, Schweiz",1,1,1,2,47.38157,8.48096,...,4.87,4.8,4.7,5.0,4.9,4.9,4.9,['Appartement war an super lage und das zimmer...,"['Föhn', 'Shampoo', 'Warmwasser', 'Duschgel', ...",0
4,Affoltern a.A. Dachwohnung/ Attic apartement,60.0,0,"Affoltern am Albis, Zürich, Schweiz",3,1,2,2,47.27565,8.45244,...,4.53,4.6,4.7,4.9,4.8,4.3,4.8,"['Alles reibungslos gelaufen!', 'Die Wohnung i...","['Föhn', 'Reinigungsprodukte', 'Shampoo', 'Sei...",0


In [ ]:
# clean comments
#todo: implement (noch offen was wir hier genau machen wollen)

In [ ]:
# clean ammeneties
#todo: split ameneties into new dataframe
# analyze which amaneties are the most common
# only use those most common ameneties (maybe top 20?)